In [37]:
## --- 2 SATELLITES DIFFERENTIAL DRAG ALGORITHM ---

from astropy import units as u

from poliastro.bodies import Earth
from poliastro.constants import rho0_earth, H0_earth
from poliastro.twobody import Orbit
from poliastro.twobody.propagation import CowellPropagator
from poliastro.core.perturbations import J2_perturbation, atmospheric_drag_exponential
from poliastro.core.propagation import func_twobody 
from poliastro.util import Time

from numba import njit as jit

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

print('DIFFERENTIAL DRAG CONTROL - 2 SATELLITES')

## 1 - INITIALIZATION

# initial orbital elements of FLOCK 1C-1 (40027) and FLOCK 1C-2 (40029) from TLEs
# Constellation parameters
T = 2

# FLOCK 1C-1:
a1, ecc1, inc1, raan1, argp1, nu1 = 6991.261179 * u.km, 0.0011698 * u.one, 97.98 * u.deg, 69.1 * u.deg, 278.7152 * u.deg, 81.2675 * u.deg 
n1 = 14.85147101   # mean motion [revolutions per day]

# FLOCK 1C-2:
a2, ecc2, inc2, raan2, argp2, nu2 = 6992.126263 * u.km, 0.0012992 * u.one, 97.98 * u.deg, 69.1 * u.deg, 275.1305 * u.deg, 84.8392 * u.deg 
n2 = 14.8487149   # mean motion [revolutions per day]

# Let's suppose the same start date (1C-1 has been selected, they differ by around 4 hours)
start_date = Time("2014-04-17 13:12:43.89", scale = "utc")

# definition of initial orbits by using poliastro
in_orbit_1 = Orbit.from_classical(Earth, a1, ecc1, inc1, raan1, argp1, nu1, start_date)
in_orbit_2 = Orbit.from_classical(Earth, a2, ecc2, inc2, raan2, argp2, nu2, start_date)

# Constants
R  = Earth.R.to(u.km).value
k  = Earth.k.to(u.km**3 / u.s**2).value
J2 = Earth.J2.value

rho0 = rho0_earth.to(u.kg/u.km**3).value
H0   = H0_earth.to(u.km).value

C_D = 2.2
LD_A_over_m = ((np.pi/4.0) * (0.01 * u.m**2) / (5 * u.kg)).to_value(u.km**2 / u.kg)   # km**2/kg
HD_A_over_m = ((np.pi/4.0) * (1 * u.m**2) / (5 * u.kg)).to_value(u.km**2 / u.kg)   # km**2/kg
LD_B = C_D * LD_A_over_m   # ballistic coefficient at low drag mode
HD_B = C_D * LD_A_over_m   # ballistic coefficient at high drag mode

DIFFERENTIAL DRAG CONTROL - 2 SATELLITES


In [38]:
in_orbit_1_state = in_orbit_1.rv()
in_orbit_2_state = in_orbit_2.rv()


unit_in_1_r = in_orbit_1_state[0] / np.linalg.norm(in_orbit_1_state[0])
unit_in_1_v = in_orbit_1_state[1] / np.linalg.norm(in_orbit_1_state[1])
unit_in_2_r = in_orbit_2_state[0] / np.linalg.norm(in_orbit_2_state[0])
unit_in_2_v = in_orbit_2_state[1] / np.linalg.norm(in_orbit_2_state[1])

dot_product_r = np.dot(unit_in_2_r, unit_in_1_r)
dot_product_v = np.dot(unit_in_2_v, unit_in_1_v)

theta     = np.arccos(dot_product_r).to_value(u.deg)
theta_dot = np.arccos(dot_product_v).to_value(u.deg)


theta_dot

0.020892355137249217

In [29]:
array_2 = np.array([*in_orbit_2_state[0].value, *in_orbit_2_state[1].value])

In [39]:
HD_acc = np.array([-0.002918724396726117, -0.007626584869733477, 4.284739870717955e-06])
LD_acc = np.array([-0.002980756990203079, -0.007583139119009421, -0.000520680685874514])

In [40]:
unit_HD_acc = HD_acc / np.linalg.norm(HD_acc)
unit_LD_acc = LD_acc / np.linalg.norm(LD_acc)

dot_product_acc = np.dot(unit_HD_acc, unit_LD_acc)
theta_dot_dot   = ((np.arccos(dot_product_acc)) * u.rad).to_value(u.deg)
theta_dot_dot

3.722456203627599

In [41]:
theta_des     = 45
theta_dot_des = 0
delta_t_hd =  (theta_dot / theta_dot_dot)
theta_hd = 0.5 * theta_dot_dot * (delta_t_hd**2)
delta_t_wait = np.absolute((theta_hd - theta_des)) / theta_dot

print(f'delta_t_wait = {delta_t_wait}\ndelta_t_hd   = {delta_t_hd}')

delta_t_wait = 2153.8951006247216
delta_t_hd   = 0.005612518722688866
